# Google Storage Read CSV with Neo4j Python API and Graph Data Science (GDS) API
In the examples that follows, we will be using the Spark Connector running under PySpark
[Neo4j gds python client](https://neo4j.com/docs/graph-data-science/current/python-client/)

Create a free account at [https://sandbox.neo4j.com](https://sandbox.neo4j.com). Choose the "Blank Sandbox - Graph Data Science" option.  When your sandbox has been created, fill in the Bolt URL and password below.

In [ ]:
## install dependencies (restart kernal after install)

In [ ]:
pip install graphdatascience

In [ ]:
pip install seaborn

In the examples that follows, we will be using the new PySpark GraphDataScience client library.

# Setup Neo4j Graph Data Science API

In [ ]:
from graphdatascience import GraphDataScience
import seaborn as sns
from matplotlib import pyplot as plt

Define Neo4j connection variables.  Yours will be different.

In [ ]:
neo4j_url = "bolt://***removed***:7687"
neo4j_user = "neo4j"
neo4j_password = "***removed***"
neo4j_database= "neo4j"

Instiantiate Graph Data Science

In [ ]:
gds = GraphDataScience(neo4j_url, auth=(neo4j_user, neo4j_password))

Check GDS version

In [ ]:
gds.run_cypher('return gds.version() as gds_version')

Reset the database if there are records

In [ ]:
gds.run_cypher("CREATE OR REPLACE DATABASE `"+neo4j_database+"`")

# Load MSA data from CSV

Start by creating a [node key constraint](https://neo4j.com/docs/cypher-manual/current/constraints/) that requires that each MSA node has a unique name.

In [ ]:
gds.run_cypher("""
CREATE CONSTRAINT msa_name IF NOT EXISTS ON (m:MSA) ASSERT m.name IS NODE KEY
""")

Now load the data. `LOAD CSV` assumes that all columns are text, so we have to cast the numeric columns to the appropriate data types.

In [ ]:
gds.run_cypher("""
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/smithna/datasets/main/CensusDemographicsByMetroArea.csv" 
AS row
WITH row WHERE row.name CONTAINS 'Metro'
MERGE (m:MSA {name:row.name})
SET m.population = toInteger(row.population),
m.medianHouseholdIncome = toInteger(row.medianHouseholdIncome),
m.medianHomePrice = toInteger(row.medianHomePrice),
m.percentOver25WithBachelors = toFloat(row.percentOver25WithBachelors)
RETURN count(m) as msaCount
""")

# Run Cypher queries for data profiling
Aggregate and find quantiles.

In [ ]:
gds.run_cypher("""
MATCH (n)
WITH n, 
["population", "medianHouseholdIncome", "medianHomePrice", "percentOver25WithBachelors" ] AS metrics
UNWIND metrics as metric
WITH metric, n[metric] AS value
RETURN metric, min(value) AS minValue,
percentileCont(value, 0.25) AS percentile25, 
percentileCont(value, 0.50) AS percentile50, 
percentileCont(value, 0.75) AS percentile75, 
max(value) AS maxValue
""")

Some of those quantiles look asymetrical. Lets plot histograms and see what happens if we apply a log transformation.
In the next section we are returning results of cypher to a data frame.

In [ ]:
msa_df = gds.run_cypher("""
MATCH (m:MSA)
RETURN m.name AS msa, 
m.population AS population,
m.medianHouseholdIncome AS medianHouseholdIncome,
m.medianHomePrice AS medianHomePrice,
m.percentOver25WithBachelors as percentOver25WithBachelors
""")

In [ ]:
fig, axes = plt.subplots(4, 2)
fig.set_size_inches(15,30)
for i in range(1,5):
    sns.histplot(msa_df.iloc[:,i], ax=axes[i-1,0])
    sns.histplot(msa_df.iloc[:, i], log_scale=True, ax=axes[i-1,1])

That log transformation looks like it will help. Run the Cypher to store the transformed values in the graph.

In [ ]:
gds.run_cypher("""
MATCH (m:MSA)
SET 
m.logPopulation = log(m.population),
m.logMedianHouseholdIncome = log(m.medianHouseholdIncome),
m.logMedianHomePrice = log(m.medianHomePrice),
m.logPercentOver25WithBachelors = log(m.percentOver25WithBachelors)
""")

# Create in-memory graph projection
Passing `"*"` as the third argument to `gds.graph.project` tells GDS to include any relationships that exist in the database in the in-memory graph. Because no relationships have been created in the graph yet, there will be no relationships in the in-memory graph projection when it is created.

In [ ]:
g_msa, result = gds.graph.project(
    'msa-graph', 
    'MSA', 
    '*', 
    nodeProperties = [
        "logPopulation", 
        "logMedianHouseholdIncome", 
        "logMedianHomePrice", 
        "logPercentOver25WithBachelors"])

Notice that when we look at the results of `gds.graph.project`, we see that the `relationshipCount` is 0.

In [ ]:
result

# Apply MinMax scalar to property values

In [ ]:
gds.alpha.scaleProperties.mutate(g_msa, 
                                 nodeProperties = [
                                     "logPopulation", 
                                     "logMedianHouseholdIncome", 
                                     "logMedianHomePrice", 
                                     "logPercentOver25WithBachelors"], 
                                 scaler = "MinMax",
                                 mutateProperty = "scaledProperties")

This next line streams node properties to the procedure caller.

In [ ]:
sp = gds.graph.streamNodeProperty(g_msa, "scaledProperties")

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(list(sp['propertyValue'])).iloc[:,0].hist()

In [ ]:
pd.DataFrame(list(sp['propertyValue'])).iloc[:,1].hist()

In [ ]:
pd.DataFrame(list(sp['propertyValue'])).iloc[:,2].hist()

In [ ]:
pd.DataFrame(list(sp['propertyValue'])).iloc[:,3].hist()

# Run KNN to create relationships to nearest neighbors
First run in stats mode and look at the similarity distribution.

In [ ]:
knn_stats = gds.knn.stats(g_msa,
                          nodeProperties={"scaledProperties":"EUCLIDEAN"},
                          topK=15
                         )

In [ ]:
knn_stats['similarityDistribution']

Now run KNN in mutate mode to update the in-memory graph projection. We'll exclude the bottom 1% of similarity relationships.

In [ ]:
gds.knn.mutate(g_msa,
               nodeProperties={"scaledProperties":"EUCLIDEAN"},
               topK=15,
               mutateRelationshipType="IS_SIMILAR",
               mutateProperty="similarity",
               similarityCutoff=knn_stats['similarityDistribution']['p1']
              )

Also write the relationships from the in-memory graph projection back to the on-disk graph.

In [ ]:
gds.graph.writeRelationship(
    g_msa,
    "IS_SIMILAR",
    relationship_property="similarity"
)

Add a `rank` property to the `IS_SIMILAR` relationships for use with Bloom filtering.

In [ ]:
gds.run_cypher("""
MATCH (m:MSA)-[s:IS_SIMILAR]->()
WITH m, s ORDER BY s.similarity DESC
WITH m, collect(s) as similarities, range(0, 19) AS ranks
UNWIND ranks AS rank
WITH rank, similarities[rank] AS rel
SET rel.rank = rank + 1
""")

# Run Louvain community detection

In [ ]:
gds.louvain.stats(g_msa,
                  relationshipTypes=["IS_SIMILAR"],
                 relationshipWeightProperty="similarity"
                 )

In [ ]:
gds.louvain.write(g_msa,
                  relationshipTypes=["IS_SIMILAR"],
                  relationshipWeightProperty ="similarity",
                  writeProperty="communityId")

# Gather statistics about the communities that were discovered

Get average values for each community and 3 example MSAs for each community.

In [ ]:
community_df = gds.run_cypher("""
MATCH (m:MSA)
WITH m 
ORDER BY apoc.coll.sum([(m)-[s:IS_SIMILAR]->(m2) 
WHERE m.communityId = m2.communityId | s.similarity]) desc
RETURN m.communityId as communityId,
count(m) as msaCount, 
avg(m.population) as avgPopulation,
avg(m.medianHomePrice) as avgHomePrice,
avg(m.medianHouseholdIncome) as avgIncome,
avg(m.percentOver25WithBachelors) as avgPctBachelors,
collect(m.name)[..3] as exampleMSAs
ORDER BY avgPopulation DESC
""")

In [ ]:
community_df.sort_values('communityId')

In [ ]:
fig, axes = plt.subplots(5, 1)
fig.set_size_inches(6,20)
for i in range(1,6):
    sns.barplot(data=community_df, x="communityId", y=community_df.columns[i], ax=axes[i-1])

Mean can give us a quick overview of properties, but can be skewed by outliers. Compare emperical cumulative distribution function (ECDF) at various proportions to get a more complete picture of distributions.

In [ ]:
detail_df = gds.run_cypher("""
MATCH (m:MSA)
RETURN "community " + m.communityId as communityId,
m.population as population,
m.medianHomePrice as medianHomePrice,
m.medianHouseholdIncome as medianIncome,
m.percentOver25WithBachelors as pctBachelors
order by m.communityId
""")

In [ ]:
fig, axes = plt.subplots(4, 1)
fig.set_size_inches(6,20)
for i in range(1,5):
    sns.ecdfplot(data=detail_df, hue="communityId", x=detail_df.columns[i], log_scale=True, ax=axes[i-1])

Compare two-dimensions on scatter plots

In [ ]:
splot = sns.scatterplot(data=detail_df, x="medianIncome", y="population", hue="communityId")
splot.set(yscale="log")
splot.set(xscale="log")

In [ ]:
splot = sns.scatterplot(data=detail_df, x="pctBachelors", y="medianHomePrice", hue="communityId")
splot.set(yscale="log")
splot.set(xscale="log")

# Assign human-friendly names to the clusters discovered.
The Louvain community detection algorithm is not deterministic. You should have roughly the same clusters from previous runs, but some edge cases might be assigned to different communities. The community numbers might be shuffled between across different runs.
**This step requires adjustment by hand: choose from community IDs above.**

In [ ]:
gds.run_cypher("""
MATCH (m:MSA) 
  SET m.communityName = CASE m.communityId 
  WHEN 54 THEN "Large mid-cost metros"
  WHEN 75 THEN "College towns"
  WHEN 81 THEN "Large high-cost metros"
  WHEN 234 THEN "Mid-size metros"
  WHEN 264 THEN "Small metros"
  WHEN 330 THEN "Mid-price metros"
  WHEN 385 THEN "Low-income metros"
  END
return m.communityName, m.communityId, count(*)
""")

Create an index on the `communityName` property to make it searchable in Bloom.

In [ ]:
gds.run_cypher("""
CREATE INDEX msa_community_name IF NOT EXISTS
FOR (m:MSA)
ON (m.communityName)
""")